In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2024-03-13 14:49:40.349922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 14:49:40.350016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 14:49:40.537619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_34/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


In [5]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [8]:
df.drop(columns=['ID'],inplace=True)

In [9]:
df.head()

,TITLE,GENRE,DESCRIPTION
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [10]:
# Get the index of the description with the maximum length
max_length_index = df['DESCRIPTION'].str.len().idxmax()

# Get the movie description with the maximum length
longest_description = df.loc[max_length_index, 'DESCRIPTION']

# Print the description and its length
print(f"Movie with the longest description:\n{longest_description}\nLength: {len(longest_description)}")


Movie with the longest description:
 Guy Gabaldon died on August 31, 2006 and the world lost someone very special. During the bloody struggle for Saipan in July 1944, U.S. Marine PFC Guy Gabaldon is indeed officially credited with capturing over 1500 Japanese soldiers and civilians - singlehandedly, a record that is untouchable in the annals of American military history. For over sixty years, Guy talked about his exploits on that island, sharing his experience and using his celebrity to inspire new generations who valued bravery and bravado. However, war experience alone does not make a life, and Guy's didn't stop in 1944. He lived many different lives and most importantly he took it upon himself to help the less fortunate, particularly the wayward teenagers he encountered when he returned to the Mariana Islands in 1980, where he would live for twenty years. Guy Gabaldon grew up in East Los Angeles where he spent more time on the streets than at home. He would get into fights and he wa

In [11]:
average_length = df['DESCRIPTION'].str.len().mean()

print(f"The average length of movie descriptions is: {average_length:.2f}")


The average length of movie descriptions is: 600.45


In [13]:
def preprocess_text(text):
    
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    # Join tokens back into a string
    preprocessed_text = ' '.join(stemmed_tokens)
    
    return preprocessed_text
df['DESCRIPTION'] = df['DESCRIPTION'].apply(preprocess_text)

In [14]:
df.drop(columns=['TITLE'],inplace=True)

In [15]:
X=df['DESCRIPTION']
y=df['GENRE']

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
print(f"The shpae of X_train is {X_train.shape}")
print(f"The shpae of X_test is {X_test.shape}")
print(f"The shpae of Y_train is {y_train.shape}")
print(f"The shpae of Y_test is {y_test.shape}")

The shpae of X_train is (43371,)
The shpae of X_test is (10843,)
The shpae of Y_train is (43371,)
The shpae of Y_test is (10843,)


In [19]:
vectorizer = TfidfVectorizer(max_features=1000)  
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
print(X_train_vectorized)
X_test_vectorized = vectorizer.transform(X_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [21]:
# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [22]:
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.8),
    Dense(256, activation='relu'),
    # Dropout(0.8),
    Dense(128, activation='relu'),
    # Dropout(0.8),

    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(27, activation='softmax')
])



optimizer = Adam(learning_rate=0.003)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_val, y_val))
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(X_test_vectorized, y_test_solution_encoded)

print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')

IndexError: tuple index out of range